# Compound shell

## Import packages

In [1]:
import pandas as pd
import numpy as np
from astropy import constants
from astronavigation.deflection import *
from astronavigation.planets import Body, SolarSystem
import numpy.linalg as LA

np.set_printoptions(precision=54)

## Useful constants

In [2]:
pc = constants.pc.to('km').value
AU = constants.au.to('km').value
c = constants.c.to('km/s').value
eps = 1/c

v_null = np.array([0, 0, 0])

# Create Solar System
ss = SolarSystem()

jupiter = ss.getPlanet('jupiter')
r_jup_arc = 20  # mean jupiter radius in arcseconds

sun = ss.getSun()

## Read and clean data

Read data

In [3]:
# path
path = 'Stars_GareqEvent2017_oneTransit_new2.dat'

# read, use delimiter='\s+' to choose every white space as delimiter
stars_data = pd.read_table(path, delimiter='\s+')

# display
pd.set_option('display.max_columns', None)
stars_data.head(10)

tAF5         tAFx               starId     etaS[deg]  sig_eta[mas]  \
0  2017.145239  2017.145239  3631075715518049024  4.475043e-01      0.099969   
1  2017.145239  2017.145239  3631485528413582336 -5.870432e-01      0.072845   
2  2017.145239  2017.145239  3631191099815174272  1.157136e-01      0.091820   
3  2017.145239  2017.145239  3631256894418771584 -1.456322e-01      0.096824   
4  2017.145239  2017.145239  3631266072764288000 -2.651681e-01      0.103025   
5  2017.145239  2017.145239  3631298199119733504 -5.711045e-01      0.076116   
6  2017.145239  2017.145239  3631241265033227392 -5.069712e-02      0.074388   
7  2017.145239  2017.145239  3631239925003381248 -1.514225e-07      0.100975   
8  2017.145239  2017.145239  3631255107712373376 -2.825577e-01      0.065251   
9  2017.145239  2017.145239  3631145740665759616  6.684247e-02      0.093621   

   zetaS[deg]  sig_zeta[mas]  etaJup[deg]  zetaJup[deg]  etaSpin[deg]  \
0    0.149847       0.372775     0.012973      0.020895     20.471557   
1    0.052752       0.312333     0.012973      0.020895     20.471557   
2    0.075482       0.339177     0.012973      0.020895     20.471557   
3   -0.074359       0.311625     0.012973      0.020895     20.471557   
4   -0.023983       0.389037     0.012973      0.020895     20.471557   
5    0.208823       0.297541     0.012973      0.020895     20.471557   
6    0.185999       0.254893     0.012973      0.020895     20.471557   
7   -0.000004       0.368185     0.012973      0.020895     20.471557   
8    0.157291       0.272657     0.012973      0.020895     20.471557   
9   -0.075513       0.311841     0.012973      0.020895     20.471557   

   zetaSpin[deg]  jupPosGCX[m]  jupPosGCY[m]  jupPosGCZ[m]   gaiaPosX[m]  \
0      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
1      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
2      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
3      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
4      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
5      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
6      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
7      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
8      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
9      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   

    gaiaPosY[m]   gaiaPosZ[m]  long_jupGC[deg]  lat_jupGC[deg]  \
0  6.053463e+10  2.630360e+10       201.426291       -7.396121   
1  6.053463e+10  2.630360e+10       201.426291       -7.396121   
2  6.053463e+10  2.630360e+10       201.426291       -7.396121   
3  6.053463e+10  2.630360e+10       201.426291       -7.396121   
4  6.053463e+10  2.630360e+10       201.426291       -7.396121   
5  6.053463e+10  2.630360e+10       201.426291       -7.396121   
6  6.053463e+10  2.630360e+10       201.426291       -7.396121   
7  6.053463e+10  2.630360e+10       201.426291       -7.396121   
8  6.053463e+10  2.630360e+10       201.426291       -7.396121   
9  6.053463e+10  2.630360e+10       201.426291       -7.396121   

   long_gaia[deg]  lat_gaia[deg]  starRA[deg]  starDec[deg]  
0       155.66443      10.151667   201.552776     -7.831722  
1       155.66443      10.151667   201.463499     -6.796371  
2       155.66443      10.151667   201.480483     -7.499324  
3       155.66443      10.151667   201.331641     -7.236695  
4       155.66443      10.151667   201.383427     -7.117590  
5       155.66443      10.151667   201.620517     -6.813578  
6       155.66443      10.151667   201.593303     -7.333806  
7       155.66443      10.151667   201.405330     -7.382970  
8       155.66443      10.151667   201.566250     -7.101704  
9       155.66443      10.151667   201.328604     -7.449173

Create queary for source_id and add parallaxes to the data

In [4]:
# not need to evaluate this cell if new data has been already downloaded
with open('query.txt', 'w') as f:
    
    star_id = stars_data['starId'].unique()
    query = 'SELECT *\nFROM gaiaedr3.gaia_source\nWHERE '
    
    for id in star_id:
        query += f'source_id = {id} '
        if id != star_id[-1]:
            query += 'OR '
    f.write(query)

In [5]:
# parallax and parallax erro are in mas
new_data = pd.read_csv('1654093971685O-result.csv')
new_data = new_data[['source_id', 'parallax', 'parallax_error']]
new_data = new_data.set_index('source_id')
new_data

parallax  parallax_error
source_id                                     
3631075715518049024   3.409062        0.032620
3631089188831339264   5.459490        0.035637
3631095407943986048   0.243898        0.018328
3631096438736162944   0.535781        0.026916
3631145740665759616   1.217448        0.033738
3631186628753669888   0.400931        0.014865
3631191099815174272   1.352386        0.018157
3631239925003381248   0.472181        0.016357
3631241265033227392   0.758436        0.020345
3631244559272698112   0.488491        0.018010
3631244937229820800   2.545210        0.030274
3631255107712373376   3.914919        0.023334
3631256894418771584   6.970485        0.017854
3631266072764288000   1.330410        0.015664
3631298199119733504   1.593033        0.015341
3631485528413582336  10.354598        0.184979

In [6]:
# add parallax
parallax = []
parallax_err = []
for id in stars_data['starId']:
    p = new_data['parallax']
    err = new_data['parallax_error']
    parallax.append(p.loc[id])
    parallax_err.append(err.loc[id])
stars_data['parallax[mas]'] = parallax
stars_data['parallax_err[mas]'] = parallax_err
stars_data

tAF5         tAFx               starId  etaS[deg]  sig_eta[mas]  \
0    2017.145239  2017.145239  3631075715518049024   0.447504      0.099969   
1    2017.145239  2017.145239  3631485528413582336  -0.587043      0.072845   
2    2017.145239  2017.145239  3631191099815174272   0.115714      0.091820   
3    2017.145239  2017.145239  3631256894418771584  -0.145632      0.096824   
4    2017.145239  2017.145239  3631266072764288000  -0.265168      0.103025   
..           ...          ...                  ...        ...           ...   
137  2017.145239  2017.145240  3631095407943986048   0.228960      0.071818   
138  2017.145239  2017.145240  3631244559272698112  -0.088713      0.049222   
139  2017.145239  2017.145240  3631244937229820800  -0.079046      0.054070   
140  2017.145239  2017.145240  3631089188831339264   0.347718      0.055859   
141  2017.145239  2017.145240  3631096438736162944   0.142424      0.046130   

     zetaS[deg]  sig_zeta[mas]  etaJup[deg]  zetaJup[deg]  etaSpin[deg]  \
0      0.149847       0.372775     0.012973      0.020895     20.471557   
1      0.052752       0.312333     0.012973      0.020895     20.471557   
2      0.075482       0.339177     0.012973      0.020895     20.471557   
3     -0.074359       0.311625     0.012973      0.020895     20.471557   
4     -0.023983       0.389037     0.012973      0.020895     20.471557   
..          ...            ...          ...           ...           ...   
137    0.058662       0.318960     0.012963      0.020873     19.823930   
138   -0.037527       0.220813     0.012963      0.020873     19.823930   
139    0.050917       0.226070     0.012963      0.020873     19.823930   
140    0.045807       0.233511     0.012963      0.020873     19.823930   
141    0.081247       0.184189     0.012963      0.020873     19.823930   

     zetaSpin[deg]  jupPosGCX[m]  jupPosGCY[m]  jupPosGCZ[m]   gaiaPosX[m]  \
0        22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
1        22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
2        22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
3        22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
4        22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
..             ...           ...           ...           ...           ...   
137      22.785597 -6.550931e+11 -2.570747e+11 -9.134985e+10 -1.338483e+11   
138      22.785597 -6.550931e+11 -2.570747e+11 -9.134985e+10 -1.338483e+11   
139      22.785597 -6.550931e+11 -2.570747e+11 -9.134985e+10 -1.338483e+11   
140      22.785597 -6.550931e+11 -2.570747e+11 -9.134985e+10 -1.338483e+11   
141      22.785597 -6.550931e+11 -2.570747e+11 -9.134985e+10 -1.338483e+11   

      gaiaPosY[m]   gaiaPosZ[m]  long_jupGC[deg]  lat_jupGC[deg]  \
0    6.053463e+10  2.630360e+10       201.426291       -7.396121   
1    6.053463e+10  2.630360e+10       201.426291       -7.396121   
2    6.053463e+10  2.630360e+10       201.426291       -7.396121   
3    6.053463e+10  2.630360e+10       201.426291       -7.396121   
4    6.053463e+10  2.630360e+10       201.426291       -7.396121   
..            ...           ...              ...             ...   
137  6.053366e+10  2.630318e+10       201.426269       -7.396111   
138  6.053366e+10  2.630318e+10       201.426269       -7.396111   
139  6.053366e+10  2.630318e+10       201.426269       -7.396111   
140  6.053366e+10  2.630318e+10       201.426269       -7.396111   
141  6.053366e+10  2.630318e+10       201.426269       -7.396111   

     long_gaia[deg]  lat_gaia[deg]  starRA[deg]  starDec[deg]  parallax[mas]  \
0        155.664430      10.151667   201.552776     -7.831722       3.409062   
1        155.664430      10.151667   201.463499     -6.796371      10.354598   
2        155.664430      10.151667   201.480483     -7.499324       1.352386   
3        155.664430      10.151667   201.331641     -7.236695       6.970485   
4       

Evaluate impact parameter

In [7]:
# star and jupiter coordinates
star = np.array([stars_data['etaS[deg]'], stars_data['zetaS[deg]']])
jup = np.array([stars_data['etaJup[deg]'], stars_data['zetaJup[deg]']])

# impact parameter in units of jupiter radius
stars_data['b_J'] = np.linalg.norm(star - jup, axis=0) * 3600 / r_jup_arc 

Evaluate parallax displacement

In [8]:
# need to transform mas to m^-1
conv = 1e-6/pc  # 1e-3/(pc*1e3)
# useful quantities
alpha = np.deg2rad(stars_data['starRA[deg]'])
delta = np.deg2rad(stars_data['starDec[deg]'])
X = stars_data['gaiaPosX[m]']
Y = stars_data['gaiaPosY[m]']
Z = stars_data['gaiaPosZ[m]']
omega = stars_data['parallax[mas]']

d_alpha = omega*(1/np.cos(delta))*(X*np.sin(alpha) - Y*np.cos(alpha)) * conv
d_delta = omega*(X*np.cos(alpha)*np.sin(delta) +Y*np.sin(alpha)*np.sin(delta) - Z*np.cos(delta)) * conv

star_gaia_dir = np.array([np.cos(alpha + d_alpha)*np.cos(delta + d_delta), np.sin(alpha + d_alpha)*np.cos(delta + d_delta), np.sin(delta + d_delta)])
star_gaia_dir = star_gaia_dir

In [9]:
# add d_alpha, d_delta
stars_data['dRA[deg]'] = np.rad2deg(d_alpha)
stars_data['dDEC[deg]'] = np.rad2deg(d_delta)

# add star_gaia_dir
stars_data['starGCdirX'] = star_gaia_dir[0]
stars_data['starGCdirY'] = star_gaia_dir[1]
stars_data['starGCdirZ'] = star_gaia_dir[2]

In [10]:
# evaluate jupiter direction

jup_gaia_dir = cartesian(stars_data['long_jupGC[deg]'], stars_data['lat_jupGC[deg]'])
jup_gaia_dir.T

array([[-0.9231431848884437 , -0.36226460444141373, -0.12872846057856285],
       [-0.9231431848884437 , -0.36226460444141373, -0.12872846057856285],
       [-0.9231431848884437 , -0.36226460444141373, -0.12872846057856285],
       [-0.9231431848884437 , -0.36226460444141373, -0.12872846057856285],
       [-0.9231431848884437 , -0.36226460444141373, -0.12872846057856285],
       [-0.9231431848884437 , -0.36226460444141373, -0.12872846057856285],
       [-0.9231431848884437 , -0.36226460444141373, -0.12872846057856285],
       [-0.9231431848884437 , -0.36226460444141373, -0.12872846057856285],
       [-0.9231431848884437 , -0.36226460444141373, -0.12872846057856285],
       [-0.9231431848884437 , -0.36226460444141373, -0.12872846057856285],
       [-0.9231431848884437 , -0.36226460444141373, -0.12872846057856285],
       [-0.9231431848884437 , -0.36226460444141373, -0.12872846057856285],
       [-0.9231431848884437 , -0.36226460444141373, -0.12872846057856285],
       [-0.92314318488844

In [11]:
# check results
jp = np.array([stars_data['jupPosGCX[m]'], stars_data['jupPosGCY[m]'], stars_data['jupPosGCZ[m]']]).T
jp_n = np.linalg.norm(jp, axis=1)

for i in range(len(jp)):
    print(jp[i]/jp_n[i])

[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.36226460444141345 -0.12872846057856285]
[-0.9231431848884438  -0.

Evaluate theta

In [12]:
# useful values
eta_s = np.deg2rad(stars_data['etaS[deg]'])
zeta_s = np.deg2rad(stars_data['zetaS[deg]'])
eta_J = np.deg2rad(stars_data['etaJup[deg]'])
zeta_J = np.deg2rad(stars_data['zetaJup[deg]'])

# gnomonic coordinates
# cos_psi = np.sin(zeta_J)*np.sin(zeta_s) + np.cos(zeta_J)*np.cos(zeta_s)*np.cos(eta_J - eta_s)
# the previous factor can be neglected because we take the fraction
x_g = np.cos(zeta_J)*np.sin(eta_J - eta_s)
y_g = np.sin(zeta_J)*np.cos(zeta_s) - np.cos(zeta_J)*np.sin(zeta_s)*np.cos(eta_J - eta_s)
theta = np.arctan(np.abs(y_g/x_g))
stars_data['theta'] = theta

In [13]:
theta

0      0.288477
1      0.053039
2      0.488380
3      0.540839
4      0.159972
         ...   
137    0.173196
138    0.521361
139    0.315617
140    0.074346
141    0.436361
Length: 142, dtype: float64

In [14]:
np.arctan(np.abs((stars_data['zetaS[deg]'] - stars_data['zetaJup[deg]'])
                                         / (stars_data['etaS[deg]'] - stars_data['etaJup[deg]'])))

0      0.288484
1      0.053043
2      0.488380
3      0.540840
4      0.159972
         ...   
137    0.173198
138    0.521361
139    0.315617
140    0.074348
141    0.436362
Length: 142, dtype: float64

Display data

In [15]:
stars_data.head(10)

tAF5         tAFx               starId     etaS[deg]  sig_eta[mas]  \
0  2017.145239  2017.145239  3631075715518049024  4.475043e-01      0.099969   
1  2017.145239  2017.145239  3631485528413582336 -5.870432e-01      0.072845   
2  2017.145239  2017.145239  3631191099815174272  1.157136e-01      0.091820   
3  2017.145239  2017.145239  3631256894418771584 -1.456322e-01      0.096824   
4  2017.145239  2017.145239  3631266072764288000 -2.651681e-01      0.103025   
5  2017.145239  2017.145239  3631298199119733504 -5.711045e-01      0.076116   
6  2017.145239  2017.145239  3631241265033227392 -5.069712e-02      0.074388   
7  2017.145239  2017.145239  3631239925003381248 -1.514225e-07      0.100975   
8  2017.145239  2017.145239  3631255107712373376 -2.825577e-01      0.065251   
9  2017.145239  2017.145239  3631145740665759616  6.684247e-02      0.093621   

   zetaS[deg]  sig_zeta[mas]  etaJup[deg]  zetaJup[deg]  etaSpin[deg]  \
0    0.149847       0.372775     0.012973      0.020895     20.471557   
1    0.052752       0.312333     0.012973      0.020895     20.471557   
2    0.075482       0.339177     0.012973      0.020895     20.471557   
3   -0.074359       0.311625     0.012973      0.020895     20.471557   
4   -0.023983       0.389037     0.012973      0.020895     20.471557   
5    0.208823       0.297541     0.012973      0.020895     20.471557   
6    0.185999       0.254893     0.012973      0.020895     20.471557   
7   -0.000004       0.368185     0.012973      0.020895     20.471557   
8    0.157291       0.272657     0.012973      0.020895     20.471557   
9   -0.075513       0.311841     0.012973      0.020895     20.471557   

   zetaSpin[deg]  jupPosGCX[m]  jupPosGCY[m]  jupPosGCZ[m]   gaiaPosX[m]  \
0      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
1      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
2      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
3      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
4      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
5      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
6      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
7      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
8      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   
9      22.787461 -6.550938e+11 -2.570753e+11 -9.135009e+10 -1.338477e+11   

    gaiaPosY[m]   gaiaPosZ[m]  long_jupGC[deg]  lat_jupGC[deg]  \
0  6.053463e+10  2.630360e+10       201.426291       -7.396121   
1  6.053463e+10  2.630360e+10       201.426291       -7.396121   
2  6.053463e+10  2.630360e+10       201.426291       -7.396121   
3  6.053463e+10  2.630360e+10       201.426291       -7.396121   
4  6.053463e+10  2.630360e+10       201.426291       -7.396121   
5  6.053463e+10  2.630360e+10       201.426291       -7.396121   
6  6.053463e+10  2.630360e+10       201.426291       -7.396121   
7  6.053463e+10  2.630360e+10       201.426291       -7.396121   
8  6.053463e+10  2.630360e+10       201.426291       -7.396121   
9  6.053463e+10  2.630360e+10       201.426291       -7.396121   

   long_gaia[deg]  lat_gaia[deg]  starRA[deg]  starDec[deg]  parallax[mas]  \
0       155.66443      10.151667   201.552776     -7.831722       3.409062   
1       155.66443      10.151667   201.463499     -6.796371      10.354598   
2       155.66443      10.151667   201.480483     -7.499324       1.352386   
3       155.66443      10.151667   201.331641     -7.236695       6.970485   
4       155.66443      10.151667   201.383427     -7.117590       1.330410   
5       155.66443      10.151667   201.620517     -6.813578       1.593033   
6       155.66443      10.151667   201.593303     -7.333806       0.758436   
7       155.66443      10.151667   201.405330     -7.382970       0.472181   
8       155.66443      10.151667   201.566250     -7.101704

### Find target stars

In [16]:
# threshold
thr = 5

# create threshold column
stars_data['thr'] = stars_data['b_J'] <= thr

# print number of targets
stars_data['thr'].value_counts()

False    133
True       9
Name: thr, dtype: int64

### Create target and reference dataframe

In [17]:
def ref_targ_split(df):
    
    filt = df['thr']

    target_df = df.loc[filt]
    reference_df = df.loc[(~filt)]
    
    return target_df, reference_df

## Evaluate deflection

In [18]:
# define epochs
epochs = stars_data['tAFx'].unique()

In [65]:
# differences
shell=[]
shell_q=[]
dt_new = []

# loop over epochs
for epoch in epochs:
    
    # take only data for the given epoch
    filt = stars_data['tAFx'] == epoch
    stars_epoch = stars_data.loc[filt]
    
    # split into target and reference
    target_df, reference_df = ref_targ_split(stars_epoch)
    
    theta_target = target_df['theta'].to_numpy()
    theta_reference = reference_df['theta'].to_numpy()
    
    bj_target = target_df['b_J'].to_numpy()
    bj_reference = reference_df['b_J'].to_numpy()
    
    
    shell_epoch = np.zeros((len(target_df), len(reference_df)))
    shell_q_epoch = np.zeros((len(target_df), len(reference_df)))
    
    
    # create body
    x_b = np.array([stars_epoch['jupPosGCX[m]'], stars_epoch['jupPosGCY[m]'], stars_epoch['jupPosGCZ[m]']]).T / 1000# in km
    
    # spin = cartesian(stars_epoch['etaSpin[deg]'].iloc[0], stars_epoch['zetaSpin[deg]'].iloc[0])
    
    body = Body(mass=jupiter.mass,
                pos=x_b[0],
                radius=jupiter.radius,
                J2=jupiter.J2,
                s=np.array([0, 0, 1])) 
    
    # sun
    x_sun = -np.array([stars_epoch['gaiaPosX[m]'], stars_epoch['gaiaPosY[m]'], stars_epoch['gaiaPosZ[m]']]).T/1000 # in km
    
    sun = Body(mass=sun.mass,
               pos=x_sun[0],
               radius=sun.radius)
    
    ##################
    #
    # print
    #
    ##################
    print(f'\n---------- epoch: {epoch} ----------\n')
    
    # loop over target
    x_targ = np.array([target_df['starGCdirX'], target_df['starGCdirY'], target_df['starGCdirZ']]).T
    for i in range(len(target_df)):
        
        # direction
        l0 = x_targ[i]
        
        # triplet
        t, n, m = on_triplet(l0, body.pos)
        
        # deflection
        dl = deflection_mod3(l0, body.pos, eps, body.mass)
        defl_tar = np.linalg.norm(dl)
        dp_eta_tar = defl_tar * np.cos(theta_target[i])
        
        # deflection quadrupole
        dp1_tar, dp2_tar = deflection_mod3(l0, body.pos, eps, body.mass, body.s, body.J2, body.radius)
        dp_q_eta_tar = dp1_tar * np.cos(theta_target[i]) + dp2_tar * np.sin(theta_target[i])
        
        # sun correction
        dl_sun = deflection_mod3(l0, sun.pos, eps, sun.mass)
        
        t_targ_new = t+(defl_tar + dp1_tar)*n + dp2_tar*m
        t_targ_new = t_targ_new/LA.norm(t_targ_new)
        
        
        ##################
        #
        # print
        #
        ##################
        print(f'\nb_J: {bj_target[i]}')
        print(f'\ndefl_tar: {np.rad2deg(defl_tar)*3600*1e6} muas')
        print(f'dp1_tar: {np.rad2deg(dp1_tar)*3600*1e6} muas')
        print(f'dp2_tar: {np.rad2deg(dp2_tar)*3600*1e6} muas')
        print(f'dp_eta_tar: {np.rad2deg(dp_eta_tar)*3600*1e6} muas')
        print(f'dp_q_eta_tar: {np.rad2deg(dp_q_eta_tar)*3600*1e6} muas\n')
        #print(f'dl_sun: {np.rad2deg(dl_sun)*3600*1e6} muas\n')
        
        
        # loop over reference
        x_ref = np.array([reference_df['starGCdirX'], reference_df['starGCdirY'], reference_df['starGCdirZ']]).T
        for j in range(len(reference_df)):
            
            # direction
            l0 = x_ref[j]
            
            # triplet
            t, n, m = on_triplet(l0, body.pos)
        
            # deflection
            dl = deflection_mod3(l0, body.pos, eps, body.mass)
            defl_ref = np.linalg.norm(dl)
            dp_eta_ref = defl_ref * np.cos(theta_reference[i])
            
            diff = dp_eta_tar - dp_eta_ref
            
            # deflection quadrupole
            dp1_ref, dp2_ref = deflection_mod3(l0, body.pos, eps, body.mass, body.s, body.J2, body.radius)
            dp_q_eta_ref = dp1_ref * np.cos(theta_reference[j]) + dp2_ref * np.sin(theta_reference[j])
            
            diff_q = dp_q_eta_tar - dp_q_eta_ref
            
            shell_epoch[i, j] = np.rad2deg(diff)*3600*1e6
            shell_q_epoch[i, j] = np.rad2deg(diff_q)*3600*1e6
            
            # sun correction
            dl_sun = deflection_mod3(l0, sun.pos, eps, sun.mass)
            
            t_ref_new = t+(defl_ref + dp1_ref)*n + dp2_ref*m
            t_ref_new = t_ref_new/LA.norm(t_ref_new)
            
            dt_new.append(np.rad2deg(np.arccos(np.dot(t_targ_new, t_ref_new)))*3600*1e6)
            
            ##################
            #
            # print
            #
            ##################
            print(f'\nb_J: {bj_reference[j]}')
            print(f'defl_ref: {np.rad2deg(defl_ref)*3600*1e6} muas')
            print(f'dp1_ref: {np.rad2deg(dp1_ref)*3600*1e6} muas')
            print(f'dp2_ref: {np.rad2deg(dp2_ref)*3600*1e6} muas')
            print(f'dp_eta_ref: {np.rad2deg(dp_eta_ref)*3600*1e6} muas')
            print(f'dp_q_eta_ref: {np.rad2deg(dp_q_eta_ref)*3600*1e6} muas\n')
            # print(f'dl_sun: {np.rad2deg(dl_sun)*3600*1e6} muas\n')
            
    
    shell.append(shell_epoch)
    shell_q.append(shell_q_epoch)


---------- epoch: 2017.1452385988 ----------


b_J: 4.427643850555587

defl_tar: 3818.160305995597 muas
dp1_tar: 1.2480085155419476 muas
dp2_tar: -2.633107797097137 muas
dp_eta_tar: 2013.7529845282845 muas
dp_q_eta_tar: -1.5788920525944625 muas


b_J: 81.58707469920164
defl_ref: 207.19541198798032 muas
dp1_ref: -0.00039352151664614197 muas
dp2_ref: -0.0002471391755680321 muas
dp_eta_ref: 198.6337488701774 muas
dp_q_eta_ref: -0.00044756974456947236 muas


b_J: 108.15506981721754
defl_ref: 156.29935766256594 muas
dp1_ref: -0.00019890086933502848 muas
dp2_ref: 2.4568423601097757e-05 muas
dp_eta_ref: 149.8408051637611 muas
dp_q_eta_ref: -0.00019731868718286482 muas


b_J: 20.941452965187587
defl_ref: 807.1806418102083 muas
dp1_ref: -0.015776277559318905 muas
dp2_ref: -0.02254487425392974 muas
dp_eta_ref: 773.8265792656581 muas
dp_q_eta_ref: -0.024509894906585276 muas


b_J: 33.30194388495801
defl_ref: 507.6331255830182 muas
dp1_ref: -0.0033251466426465763 muas
dp2_ref: -0.0059916734895215


b_J: 81.58860576694823
defl_ref: 207.1910502054816 muas
dp1_ref: -0.0003934844270057592 muas
dp2_ref: -0.0002471430544316954 muas
dp_eta_ref: 198.6286964757671 muas
dp_q_eta_ref: -0.0004475371375230152 muas


b_J: 108.15405940427665
defl_ref: 156.30091409929162 muas
dp1_ref: -0.00019890567553121629 muas
dp2_ref: 2.457834899366422e-05 muas
dp_eta_ref: 149.84164033496353 muas
dp_q_eta_ref: -0.00019732212774386298 muas


b_J: 20.943924207029088
defl_ref: 807.0986415374343 muas
dp1_ref: -0.015767968253263945 muas
dp2_ref: -0.02254045408089109 muas
dp_eta_ref: 773.7445750526035 muas
dp_q_eta_ref: -0.024501807396938573 muas


b_J: 33.29967887114859
defl_ref: 507.6674191680099 muas
dp1_ref: -0.0033263650911188112 muas
dp2_ref: -0.005992585670278233 muas
dp_eta_ref: 486.68761325623535 muas
dp_q_eta_ref: -0.00593676654571101 muas


b_J: 50.711518449008324
defl_ref: 333.3409321927989 muas
dp1_ref: -0.0018529997972948694 muas
dp2_ref: -0.0005788885442146236 muas
dp_eta_ref: 319.5653228158646 mua

In [51]:
#shell

In [52]:
#shell_q

Create data

In [71]:
# angular separation
data_y = []
for epoch in epochs:
    
    print(f'\n---------- epoch: {epoch} ----------\n')
    
    # take only data for the given epoch
    filt = stars_data['tAFx'] == epoch
    stars_epoch = stars_data.loc[filt]
    
    # split into target and reference
    target_df, reference_df = ref_targ_split(stars_epoch)
    
    theta_target = target_df['theta'].to_numpy()
    theta_reference = reference_df['theta'].to_numpy()
    
    r_p = np.zeros((len(target_df), len(reference_df)))
    r_m = np.zeros((len(target_df), len(reference_df)))
    
    err_r_p = np.zeros((len(target_df), len(reference_df)))
    err_r_m = np.zeros((len(target_df), len(reference_df)))
    
    eta_targ_p = target_df['etaS[deg]'].to_numpy()
    eta_ref_p = reference_df['etaS[deg]'].to_numpy()
    
    zeta_targ_p = target_df['zetaS[deg]'].to_numpy()
    zeta_ref_p = reference_df['zetaS[deg]'].to_numpy()
    
    err_eta_targ = target_df['sig_eta[mas]'].to_numpy() * 1e3
    err_eta_ref = reference_df['sig_eta[mas]'].to_numpy() * 1e3
    
    err_zeta_targ = target_df['sig_zeta[mas]'].to_numpy() * 1e3
    err_zeta_ref = reference_df['sig_zeta[mas]'].to_numpy() * 1e3
    
    x_targ = target_df['starGCdirX'].to_numpy()
    y_targ = target_df['starGCdirY'].to_numpy()
    z_targ = target_df['starGCdirZ'].to_numpy()
    
    x_ref = reference_df['starGCdirX'].to_numpy()
    y_ref = reference_df['starGCdirY'].to_numpy()
    z_ref = reference_df['starGCdirZ'].to_numpy()
    
    alpha_targ = target_df['starRA[deg]'].to_numpy() + target_df['dRA[deg]'].to_numpy()
    delta_targ = target_df['starDec[deg]'].to_numpy() + target_df['dDEC[deg]'].to_numpy()
    alpha_ref =  reference_df['starRA[deg]'].to_numpy() +  reference_df['dRA[deg]'].to_numpy()
    delta_ref = reference_df['starDec[deg]'].to_numpy() +  reference_df['dDEC[deg]'].to_numpy()
    
    jup_epoch = np.array([stars_epoch['jupPosGCX[m]'], stars_epoch['jupPosGCY[m]'], stars_epoch['jupPosGCZ[m]']]).T
    jup_epoch = jup_epoch[0]
    jup_epoch = jup_epoch/LA.norm(jup_epoch)
    
    # loop over target
    for i in range(len(target_df)):
        
        r_targ = np.array([x_targ[i], y_targ[i], z_targ[i]]) 
        
        chi_targ = np.rad2deg(np.arccos(np.dot(r_targ, jup_epoch))) *3600*1e6
        chi_targ_eta = chi_targ*np.cos(theta_target[i])
        #print(f'chi_targ:       {chi_targ}')
        #print(f'chi_targ_eta:   {chi_targ_eta}')
        
        # loop over reference
        for j in range(len(reference_df)):
            
            r_ref = np.array([x_ref[j], y_ref[j], z_ref[j]])
            
            theta_rel = np.arctan(np.abs((zeta_ref_p[j] - zeta_targ_p[i]) /  (eta_ref_p[j] - eta_targ_p[i])))

            
            chi_ref = np.rad2deg(np.arccos(np.dot(r_ref, jup_epoch))) *3600*1e6
            chi_ref_eta = chi_ref*np.cos(theta_reference[j])
            
            #print(f'chi_ref:      {chi_ref}')
            #print(f'chi_ref_eta:  {chi_ref_eta}')
            
            #r_p[i, j] = np.sqrt((eta_targ_p[i] - eta_ref_p[j])**2 + (zeta_targ_p[i] - zeta_ref_p[j])**2)*3600*1e6
            #r_p[i, j] = np.abs((eta_targ_p[i] - eta_ref_p[j])) *3600*1e6
            r_p[i, j] = np.rad2deg(np.arccos(np.dot(cartesian(eta_targ_p[i], zeta_targ_p[i]), cartesian(eta_ref_p[j], zeta_ref_p[j]))))*3600*1e6
            r_m[i, j] = np.rad2deg(np.arccos(np.dot(r_targ, r_ref))) *3600*1e6 #* np.cos(theta_rel)
            # r_m [i, j] = np.sqrt((alpha_targ[i] - alpha_ref[j])**2 + (delta_targ[i] - delta_ref[j])**2)*3600*1e6
            #r_m[i, j] = chi_ref_eta + chi_targ_eta
            
            
            err_r_p[i, j] = LA.norm([err_eta_targ[i], err_eta_ref[j]])
            
            print(f'sep_j: {r_p[i, j]} muas')
            print(f'sep:   {r_m[i, j]} muas\n')
            # print(f'err: {err_r_p[i, j]} muas\n')
            
             
                
    data_y.append(np.abs(r_p ))



---------- epoch: 2017.1452385988 ----------

sep_j: 1698937361.5648944 muas
sep:   1699023665.0263872 muas

sep_j: 2121871202.5462296 muas
sep:   2121954619.0187488 muas

sep_j: 497370222.5763862 muas
sep:   497419076.0234834 muas

sep_j: 588656613.5339639 muas
sep:   588667389.1310484 muas

sep_j: 958500091.284178 muas
sep:   958565057.0145519 muas

sep_j: 2189105287.5056744 muas
sep:   2189219984.2119365 muas

sep_j: 694034943.6181889 muas
sep:   694076358.7459843 muas

sep_j: 1164199740.762505 muas
sep:   1164279126.4381373 muas

sep_j: 363040442.2334474 muas
sep:   363075162.3847944 muas

sep_j: 936879861.9953325 muas
sep:   936925918.3095059 muas

sep_j: 850879654.7142038 muas
sep:   850943149.5677729 muas

sep_j: 346763651.87507296 muas
sep:   346786064.8158462 muas

sep_j: 338495023.42913234 muas
sep:   338535457.8588764 muas

sep_j: 1262604211.7062619 muas
sep:   1262671035.275076 muas

sep_j: 590290586.0975205 muas
sep:   590340847.0888053 muas


---------- epoch: 2017.14523

In [54]:
data_y

[array([[ 80185.76462817192 ,  82785.50611543655 ,  40832.38237071037 ,
           9480.320859968662,  64674.04685115814 , 103706.36916661263 ,
          10868.687897533178,  68386.94259691238 ,  22982.886766254902,
          34984.76634120941 ,  61372.87426543236 ,  20622.791236639023,
          33965.858401179314,  66119.90243196487 ,  43526.996039032936]]),
 array([[83003.4301366806  , 81508.14464616776 , 39467.64706802368 ,
          8668.12893640995 , 64102.227576971054, 99914.1778473854  ,
         10547.383062541485, 66358.79535746574 , 21992.338634848595,
         30582.247418284416, 56340.57581484318 , 20951.12184536457 ,
         31178.268181681633, 66381.74033784866 , 43890.986949265   ]]),
 array([[82633.72802853584 , 81774.91015124321 , 33643.73515552282 ,
          9704.066941201687, 64360.547201275826, 99542.48577833176 ,
         10357.831531077623, 66013.6342587471  , 22513.705278545618,
         26924.1788469553  , 55791.69803619385 , 22035.13167345524 ,
         3095

In [55]:
x_test_s = cartesian(stars_data['starRA[deg]'], stars_data['starDec[deg]']).T
par = stars_data['parallax[mas]'].to_numpy() * conv
x_test_s = np.array([x_test_s[i]*(1/par[i]) for i in range(len(par))])

In [56]:
x_gaia = np.array([stars_data['gaiaPosX[m]'], stars_data['gaiaPosY[m]'], stars_data['gaiaPosZ[m]']]).T
x_s_GC = x_test_s - x_gaia

In [57]:
x_s_GC_n = np.linalg.norm(x_s_GC, axis=1)
for i in range(len(x_s_GC)):
    print(x_s_GC[i]/x_s_GC_n[i])

[-0.9214043052868125  -0.36393159456468027 -0.13626408432354978]
[-0.9241111718940843  -0.3633372096341643  -0.11834109208470911]
[-0.9225828691221889  -0.3630520857591138  -0.13051449202378737]
[-0.9240704342155827  -0.3608680406623014  -0.12596860655407793]
[-0.9239857368449542  -0.36179777605462515 -0.12390612312962657]
[-0.9230789168676675  -0.36585521137984095 -0.11863927486556017]
[-0.9222129300578042  -0.3650052467429338  -0.12764983934314025]
[-0.9233031401324711 -0.3619376259516455 -0.1285008417949288]
[-0.9228584994264368  -0.36475686184345407 -0.1236309903482972 ]
[-0.9236480630635924  -0.3606466489889742  -0.12964663579089825]
[-0.9218292427336883  -0.3644252955750325  -0.13200398170542046]
[-0.9224548069178967 -0.3626685019176736 -0.1324714569666093]
[-0.9237219884372454  -0.3614124309099047  -0.12695961114202683]
[-0.9231373726598383  -0.36283995069032365 -0.12713992835350432]
[-0.9222858822683673  -0.36234254345416567 -0.13452372493960094]
[-0.9224906063954232  -0.363119

In [58]:
np.array([stars_data['starGCdirX'], stars_data['starGCdirY'], stars_data['starGCdirZ']]).T[0]

array([-0.9214043052868126 , -0.36393159456468027, -0.13626408432354978])

## Non linear fit

In [59]:
from scipy.optimize import curve_fit

In [60]:
def model(x_data, gamma, epsilon):
    
    dpsi_mono = x_data[0]
    dpsi_quad = x_data[1]
    
    dr = (1+gamma)*(dpsi_mono + epsilon*dpsi_quad)
    
    return dr

In [66]:
shell = [ep.squeeze() for ep in shell]
shell = np.concatenate(shell)

shell_q = [ep.squeeze() for ep in shell_q]
shell_q = np.concatenate(shell_q)

data_y = [ep.squeeze() for ep in data_y]
data_y = np.concatenate(data_y)

In [67]:
xdata = np.array([shell/2, shell_q/2]).squeeze()
ydata = np.array(data_y).squeeze()
ydata

array([ 80185.76462817192 ,  82785.50611543655 ,  40832.38237071037 ,
         9480.320859968662,  64674.04685115814 , 103706.36916661263 ,
        10868.687897533178,  68386.94259691238 ,  22982.886766254902,
        34984.76634120941 ,  61372.87426543236 ,  20622.791236639023,
        33965.858401179314,  66119.90243196487 ,  43526.996039032936,
        83003.4301366806  ,  81508.14464616776 ,  39467.64706802368 ,
         8668.12893640995 ,  64102.227576971054,  99914.1778473854  ,
        10547.383062541485,  66358.79535746574 ,  21992.338634848595,
        30582.247418284416,  56340.57581484318 ,  20951.12184536457 ,
        31178.268181681633,  66381.74033784866 ,  43890.986949265   ,
        82633.72802853584 ,  81774.91015124321 ,  33643.73515552282 ,
         9704.066941201687,  64360.547201275826,  99542.48577833176 ,
        10357.831531077623,  66013.6342587471  ,  22513.705278545618,
        26924.1788469553  ,  55791.69803619385 ,  22035.13167345524 ,
        30956.886102

In [32]:
rng = np.random.default_rng()
y_noise = 0.05 * rng.normal(size=xdata.shape[1])
#ydata = np.array(shell) + np.array(shell_q) +y_noise
#ydata = ydata.squeeze()
ydata

array([1813.4717360216566, 1862.2808073488286, 1238.3748020237315,
       1525.5796307248352, 1692.5284968440687, 1865.4970742749417,
       1503.352949486423 , 1735.587780266049 , 1196.9792626011354,
       1632.4237929052308, 1601.660256547978 , 1244.4538050743777,
       1082.1305418937422, 1744.008856028367 , 1381.863071141698 ,
       1813.7423408138493, 1862.5764251253618, 1238.6419006332094,
       1525.7982773089523, 1692.8298200568818, 1865.68728854875  ,
       1503.7492120492013, 1735.6780476848262, 1197.1531525157866,
       1632.7970858276203, 1601.7795574078941, 1244.7099638075597,
       1082.3442791228806, 1744.2157483810215, 1382.1051019492704,
       1814.0774905168146, 1862.793747031205 , 1238.79690336823  ,
       1526.0257343881387, 1693.1773615912416, 1865.9199489284924,
       1503.95500616053  , 1736.0543840801272, 1197.3716165712603,
       1632.889059828045 , 1602.0530789234622, 1245.0065940119043,
       1082.504566140864 , 1744.450445916808 , 1382.3252241809

In [68]:
# constrain optimization
popt, pcov = curve_fit(model, xdata, ydata, p0=(1,0), bounds=(0, [10, 10]))

In [69]:
popt

array([9.999999999990729e+00, 7.453871181014125e-20])

In [72]:
data_y

[array([[1.6989373615648944e+09, 2.1218712025462296e+09,
         4.9737022257638621e+08, 5.8865661353396392e+08,
         9.5850009128417802e+08, 2.1891052875056744e+09,
         6.9403494361818886e+08, 1.1641997407625051e+09,
         3.6304044223344737e+08, 9.3687986199533248e+08,
         8.5087965471420383e+08, 3.4676365187507296e+08,
         3.3849502342913234e+08, 1.2626042117062619e+09,
         5.9029058609752047e+08]]),
 array([[1.6989343903110361e+09, 2.1218724850330541e+09,
         4.9737185185914820e+08, 5.8865752545387316e+08,
         9.5850066543930221e+08, 2.1891093250141973e+09,
         6.9403616550546193e+08, 1.1642020618871789e+09,
         3.6304193634449083e+08, 9.3688555995521140e+08,
         8.5088484918637443e+08, 3.4676329545291686e+08,
         3.3849833901304448e+08, 1.2626039476053123e+09,
         5.9029016702782869e+08]]),
 array([[1.6989347802121346e+09, 2.1218722171949961e+09,
         4.9737880491244715e+08, 5.8865636233788311e+08,
         9.58500

In [73]:
dt_new

[1699026814.5529072,
 2121953056.9490361,
 497423341.3206531,
 588664624.6387115,
 958563088.7437317,
 2189219356.823698,
 694079485.6258152,
 1164279229.4374106,
 363074901.56809795,
 936929931.6748931,
 850946335.3846494,
 346783738.6123671,
 338536457.4527561,
 1262673737.903453,
 590344861.4741409,
 1699026814.9449055,
 2121953056.6908162,
 497423341.7384713,
 588664624.2134372,
 958563088.413578,
 2189219356.672663,
 694079485.8844193,
 1164279229.3562706,
 363074901.49004024,
 936929932.128623,
 850946335.7565564,
 346783738.2037437,
 338536457.41089827,
 1262673738.2363899,
 590344861.8822023,
 1699026815.331344,
 2121953056.432596,
 497423342.16578543,
 588664623.8122351,
 958563088.0982075,
 2189219356.521628,
 694079486.1566343,
 1164279229.2751303,
 363074901.43800175,
 936929932.5773115,
 850946336.1173619,
 346783737.79512024,
 338536457.3829931,
 1262673738.573068,
 590344862.306266,
 1699026815.7261221,
 2121953056.1766014,
 497423342.5930996,
 588664623.3869609,
 958563